# Using CleanRL PPO to try and train Villagers

In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import raw_env
import random
from tqdm import tqdm

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Slightly based on [this](https://pettingzoo.farama.org/tutorials/cleanrl/implementing_PPO/), and the [following blogpost](https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/). Another PZ implementation referenced is [here](https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/ppo_pettingzoo_ma_atari.py)

One more link I plan on reading, at least for PPO is [here](https://towardsdatascience.com/elegantrl-mastering-the-ppo-algorithm-part-i-9f36bc47b791)

In [2]:
env = raw_env(num_agents=10, werewolves=2)
env.reset()

In [3]:
num_actions = env.action_spaces['player_1'].n
observation_size = env.observation_spaces['player_1']['observation']

In [4]:
env.observation_spaces['player_1']['observation'].shape

In [4]:
env.observation_spaces['player_1'].sample()

OrderedDict([('action_mask',
              array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
                      True])),
             ('observation',
              OrderedDict([('day', 1),
                           ('phase', 2),
                           ('player_status',
                            array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
                                    True])),
                           ('roles', array([1, 1, 0, 1, 0, 0, 1, 1, 1, 1])),
                           ('votes',
                            array([10.892954 ,  0.8731968,  8.033873 ,  2.277371 ,  1.8526876,
                                   10.99731  ,  7.982339 ,  5.003656 ,  2.5943236,  2.754004 ],
                                  dtype=float32))]))])

In [7]:
env.observation_spaces['player_1'].sample()['observation'].values()

odict_values([3, 2, array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True]), array([0, 1, 0, 0, 0, 1, 1, 1, 0, 0]), array([3.8632665, 1.7954935, 6.4708276, 9.651167 , 7.759436 , 2.4337952,
       3.148949 , 1.946332 , 4.701349 , 4.30324  ], dtype=float32)])

In [38]:
def flat_obs(observation):
    return  np.asarray([observation['day']] + \
            [observation['phase']] + \
            [int(status) for status in observation['player_status']] + \
            [role for role in observation['roles']] + \
            [vote for vote in observation['votes']])

In [44]:
flat_obs(env.observation_spaces['player_1'].sample()['observation']).shape[-1]

32

## PPO Training

In [ ]:
class Agent(torch.nn.Module):
    def __init__(self, num_actions):
        super().__init__()

        self.critic = torch.nn.Sequential(
            self._layer_init(torch.nn.Linear(obs_size, 64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,1), std=1.0),
        )

        self.actor = torch.nn.Sequential(
            self._layer_init(torch.nn.Linear(obs_size, 64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64,64)),
            torch.nn.Tanh(),
            self._layer_init(torch.nn.Linear(64, num_actions), std=0.01),
        )
    
    def get_value(self, x):
        return self.critic(x)
    
    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)

        probs = torch.distributions.categorical.Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)

    def _layer_init(self, layer, std=np.sqrt(2), bias_const=0.0):
        torch.nn.orthogonal_(layer.weight, std)
        torch.nn.constant_(layer.bias, bias_const)
        return layer

def batchify_obs(obs, device):
    """Converts PZ style observations to batch of torch arrays."""
    # convert to list of np arrays
    obs = np.stack([obs[a] for a in obs], axis=0)
    obs = torch.tensor(obs).to(device)

def batchify(x, device):
    """Converts PZ style returns to batch of torch arrays."""
    # convert to list of np arrays
    x = np.stack([x[a] for a in x], axis=0)
    # convert to torch
    x = torch.tensor(x).to(device)

    return x

def unbatchify(x, env):
    """Converts np array to PZ style arguments."""
    x = x.cpu().numpy()
    x = {a: x[i] for i, a in enumerate(env.possible_agents)}

    return x


In [ ]:
# ALGORITHM PARAMETERS
# TODO: What is really necessary here?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ent_coef = 0.1 #
vf_coef = 0.1 #
clip_coef = 0.1 #
gamma = 0.99 #
batch_size = 32 #
stack_size = 4 #
frame_size = (64, 64) #
max_cycles = 125 #
total_episodes = 2 #

In [35]:
### Env Setup
env = raw_env(num_agents=10, werewolves=2)
# env.reset()
num_agents = 10
num_actions = env.action_spaces['player_1'].n
observation_size = flat_obs(env.observation_spaces['player_1'].sample()['observation']).shape[-1]

# Learner Setup
agent = Agent(num_actions=num_actions).to(device)
optimizer = torch.optim.Adam(agent.parameters(), lr=0.001, eps=1e-5)

# Algorithm Logic : Episode Storage

# rb = rollback
end_step = 0
total_episodic_return = 0
rb_obs = 
rb_actions =
rb_logprobs = 
rb_rewards = 
rb_terms = 
rb_values =


In [ ]:
# Training Logic
for episode in range(total_episodes):
    

In [ ]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")